In [ ]:
!pip install -qU langchain-pinecone pinecone-notebooks

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores.pinecone import Pinecone
from dotenv import load_dotenv

load_dotenv()
loader = Docx2txtLoader('소득세법(법률)(제19933호)(20240701).docx')
documents = loader.load()

In [ ]:
# 문서 쪼개기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)
split_documents = text_splitter.split_documents(documents)

In [ ]:
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings( #← OpenAIEmbeddings를 초기화
    model="embedding-query") #← 모델명을 지정

In [ ]:
vector_store = Pinecone.from_documents(
    documents=split_documents,
    embedding=embeddings,
    index_name = 'upstage-tax-1500'
)

In [ ]:
vectorstore_docs = vector_store.similarity_search('종합소득세', k = 4)
vectorstore_docs

# 답변 얻기

In [ ]:
from langchain.chains import RetrievalQA
from langchain_upstage import UpstageEmbeddings, ChatUpstage
from langchain.vectorstores.pinecone import Pinecone

llm = ChatUpstage(model = 'solar-pro')
embeddings = UpstageEmbeddings(model = 'embedding-query')
load_vec_db = Pinecone.from_existing_index(
    index_name='upstage-tax-1500',
    embedding=embeddings
)

retriever = load_vec_db.as_retriever()

qa_retriever = RetrievalQA.from_llm(
    llm = llm,
    retriever = retriever
)

qa_retriever.invoke('연봉 3500만원인 직장인의 종합소득세는?')